In [1]:
import cv2

from skimage import morphology
from skimage.measure import label, regionprops
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import skimage
import time
import pyvista as pv

In [2]:
def padd_to_proper(data):
    padded_image = np.pad(data[:,:,:], ((2,2), (1,1), (1, 1)), mode='constant') 
    return padded_image

In [3]:
data_path ='data/registered/output/fixed_low_res_5.tif'
data = tifffile.imread(data_path)
target_path = 'data/registered/output/registered_mask_5.tif'
target = tifffile.imread(target_path)

UNET_pred_path = 'segmentation_outputs/3D_Unet_results.tif'
UNET_pred = tifffile.imread(UNET_pred_path)

NGMM_path = 'segmentation_outputs/NGMM.tif'
NGMM_pred = tifffile.imread(NGMM_path)
NGMM_pred = padd_to_proper(NGMM_pred)
RW_path = 'segmentation_outputs/RW.tif'
RW_pred = tifffile.imread(RW_path)

GrabCut_path = 'segmentation_outputs/GrabCut.tif'
GrabCut_pred = tifffile.imread(GrabCut_path)

# cropping a volume to make sure all the colume contains the specimen
data = data [1:-2,40:-50,0:-25]
target = target[1:-2,40:-50,0:-25]
UNET_pred =UNET_pred[1:-2,40:-50,0:-25]



print ('data shape = ',data.shape)
print ('target shape = ',target.shape)
print ('Unet Pred shape = ', UNET_pred.shape)
print ('NGMMPred shape = ', NGMM_pred.shape)
print ('RW_Pred shape = ', RW_pred.shape)
print ('GrabCut_pred shape = ', GrabCut_pred.shape)






data shape =  (125, 166, 103)
target shape =  (125, 166, 103)
Unet Pred shape =  (125, 166, 103)
NGMMPred shape =  (125, 166, 103)
RW_Pred shape =  (125, 166, 103)
GrabCut_pred shape =  (125, 166, 103)


In [4]:
def get_coords (IMAGE):
    points=[]
    # Getting labels and regions
    # ice is background so background intensity is 1
    labelled= morphology.label(IMAGE, connectivity=2, background=1)
    regions = regionprops(labelled)
    sorted_regions = sorted(regions, key=lambda x: x.area)
    #regions = sorted_regions[2:]
    for r in regions:
        points.append(r.coords)
        
    return points


In [7]:
def plot_3D (pred, real):   
    
    #getting coordinates of objects
    real = get_coords(real)
    print (len(real), 'real objects')
    
    pred = get_coords(pred)
    print (len(pred), 'pred objects')

    
    # Create a plotter
    plotter = pv.Plotter(off_screen = True,image_scale = 1)
    
    for p in real:
        poly_data1 = pv.PolyData(p)
        mesh1 = poly_data1.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh1, color='white', opacity=0.6)
    
    print ('plotting real is done')
    
    for j in pred[1:]:
        poly_data2 = pv.PolyData(j)
        mesh2 = poly_data2.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh2, color='blue', opacity=0.6)
        
    print ('plotting pred is done')
    

    #plotter.view_isometric()
    #plotter.show_grid()
    #plotter.show()
    plotter.export_obj('NGMM_Pred.obj')
    
plot_3D(NGMM_pred[4:-4,4:-4,4:-4], target[4:-4,4:-4,4:-4])

280 real objects
112 pred objects


C:\Users\Fabagh001\Anaconda3\envs\FARAMARZ\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


plotting real is done
plotting pred is done


In [11]:
def plot_3D (pred, real):   
    
    #getting coordinates of objects
    real = get_coords(real)
    print (len(real), 'real objects')
    
    pred = get_coords(pred)
    print (len(pred), 'pred objects')

    
    # Create a plotter
    plotter = pv.Plotter(off_screen = True,image_scale = 1)
    
    for p in real:
        poly_data1 = pv.PolyData(p)
        mesh1 = poly_data1.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh1, color='white', opacity=0.6)
    
    print ('plotting real is done')
    
    for j in pred[1:]:
        poly_data2 = pv.PolyData(j)
        mesh2 = poly_data2.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh2, color='blue', opacity=0.6)
        
    print ('plotting pred is done')
    

    #plotter.view_isometric()
    #plotter.show_grid()
    #plotter.show()
    plotter.export_obj('RW_Pred.obj')
    
plot_3D(RW_pred, target)

341 real objects
108 pred objects
plotting real is done
plotting pred is done


In [8]:
def plot_3D (pred, real):   
    
    #getting coordinates of objects
    real = get_coords(real)
    print (len(real), 'real objects')
    
    pred = get_coords(pred)
    print (len(pred), 'pred objects')

    
    # Create a plotter
    plotter = pv.Plotter(off_screen = True,image_scale = 1)
    
    for p in real:
        poly_data1 = pv.PolyData(p)
        mesh1 = poly_data1.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh1, color='white', opacity=0.6)
    
    print ('plotting real is done')
    
    for j in pred[1:]:
        poly_data2 = pv.PolyData(j)
        mesh2 = poly_data2.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh2, color='blue', opacity=0.6)
        
    print ('plotting pred is done')
    

    #plotter.view_isometric()
    #plotter.show_grid()
    #plotter.show()
    plotter.export_obj('GrabCut_pred.obj')
    
plot_3D(GrabCut_pred, target)

341 real objects
113 pred objects
plotting real is done
plotting pred is done


In [24]:
def plot_3D (pred, real):   
    
    #getting coordinates of objects
    real = get_coords(real)
    print (len(real), 'real objects')
    
    train = pred.copy()
    train[:31] = 1
    pred_train = get_coords(train)
    print (len(pred_train), 'pred_train objects')

    pred_test = get_coords(pred[:30])
    print (len(pred_test), 'pred_test objects')

    
    # Create a plotter
    plotter = pv.Plotter(off_screen = True,image_scale = 1)
    
    for p in real:
        poly_data1 = pv.PolyData(p)
        mesh1 = poly_data1.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh1, color='white', opacity=0.6)
    
    print ('plotting real is done')
    
    for j in pred_train[:]:
        poly_data2 = pv.PolyData(j)
        mesh2 = poly_data2.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh2, color='blue', opacity=0.6)
    
    for j in pred_test[:]:
        poly_data2 = pv.PolyData(j)
        mesh2 = poly_data2.delaunay_3d(alpha=4)
        plotter.add_mesh(mesh2, color='red', opacity=0.6)
        
    print ('plotting pred is done')
    

    #plotter.view_isometric()
    #plotter.show_grid()
    #plotter.show()
    plotter.export_obj('New_UNET_Pred.obj')
    
plot_3D(UNET_pred, target)

341 real objects
133 pred_train objects
56 pred_test objects
plotting real is done
plotting pred is done
